In [ ]:
# Basic Packages
import numpy as np
import h5py
import logging
import os
import shutil
import gc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress, pearsonr, spearmanr
import seaborn as sns
from scipy.optimize import curve_fit
import scipy.stats as stats
# Physics-related Packages
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u
from astropy.constants import M_sun
import ast

# extra pack for X-ray mocking
from regions import RectangleSkyRegion
from astropy.coordinates import SkyCoord

from astropy import wcs
from astropy.io import fits
import yt
import pyxsim
from matplotlib.colors import LogNorm


from scipy.ndimage import gaussian_filter
from copy import deepcopy
from astropy.wcs.utils import skycoord_to_pixel
from astropy.visualization import simple_norm

In [ ]:
# get GroupPos and GroupR
# read CSV
cat_df_file = '/users_path/merger_trace/data/tng_cluster/tng_cluster_products/fof_halo_to_sub84.csv'
cat_df = pd.read_csv(cat_df_file)

cat_df['GroupPos'] = cat_df['GroupPos'].apply(ast.literal_eval)

FOF_Halo_IDs = cat_df['FOF_Halo_ID_At84']
GroupPos = cat_df['GroupPos']
Group_R_Crit500 = cat_df['Group_R_Crit500']
SubhaloIDs = cat_df['Subhalo_ID_At84']

In [ ]:
# get sim info
import requests
import time

headers = {"api-key": "API KEY"}


# get sim info
snapshot = 84
sim_url = "http://www.tng-project.org/api/TNG-Cluster/"
snap_url = f"http://www.tng-project.org/api/TNG-Cluster/snapshots/{snapshot}/"

r = requests.get(sim_url, headers=headers)
sim_info = r.json()

snap = requests.get(snap_url, headers=headers)
snap_info = snap.json()

redshift = snap_info['redshift']
factor_a = 1/(1+redshift)

In [ ]:
def Read_imgf(halo_index):
    halo_id = int(FOF_Halo_IDs[halo_index])
    img_file_name = img_basic_path + f'halo_{halo_id}_img.fits'

    with fits.open(img_file_name) as hdul:
        hdr = hdul[0].header
        img = hdul[0].data.astype(float) 
    
    return hdr, img

In [ ]:
def Get_Regions(redshift, halo_index):
    z       = redshift
    R_phys  = Group_R_Crit500[halo_index]/(factor_a*sim_info['hubble']) * u.kpc 
    center  = SkyCoord(0*u.deg, 0*u.deg, frame="icrs")
    DA      = cosmo.angular_diameter_distance(z)  
    theta   = (R_phys / DA).to(u.dimensionless_unscaled) * u.rad
    theta_deg = theta.to(u.deg)

    offsets = [(+theta_deg, +theta_deg), (-theta_deg, +theta_deg),
           (-theta_deg, -theta_deg), (+theta_deg, -theta_deg)]
    corners = [center.spherical_offsets_by(dra, ddec) for dra, ddec in offsets]

    for i, c in enumerate(corners,1):
        print(f"C{i}: RA={c.ra.wrap_at(180*u.deg):.4f}  Dec={c.dec:.4f}")

    ra_vals  = [c.ra.wrap_at(180*u.deg).deg for c in corners]
    dec_vals = [c.dec.deg for c in corners]
    print(f"RA  span: {min(ra_vals):.4f} – {max(ra_vals):.4f} deg")
    print(f"Dec span: {min(dec_vals):.4f} – {max(dec_vals):.4f} deg")

    ra_min = min(ra_vals)
    ra_max = max(ra_vals)
    dec_min = min(dec_vals)
    dec_max = max(dec_vals)

    return corners, ra_min, ra_max, dec_min, dec_max

In [ ]:
def Crop_img(hdr, img, redshift, halo_index):
    corners, ra_min, ra_max, dec_min, dec_max = Get_Regions(redshift, halo_index)

    corners_sky = SkyCoord([ra_min, ra_max, ra_max, ra_min]*u.deg,
                        [dec_min, dec_min, dec_max, dec_max]*u.deg,
                        frame='icrs')
    x_pix, y_pix = skycoord_to_pixel(corners_sky, wcs.WCS(hdr))

    
    xmin, xmax = int(x_pix.min()), int(x_pix.max())+1
    ymin, ymax = int(y_pix.min()), int(y_pix.max())+1

    # crop data
    img_crop = img[ymin:ymax, xmin:xmax]

    hdr_crop = deepcopy(hdr)
    hdr_crop['NAXIS1'], hdr_crop['NAXIS2'] = img_crop.shape[1], img_crop.shape[0]
    hdr_crop['CRPIX1'] -= xmin     
    hdr_crop['CRPIX2'] -= ymin

    # fits.writeto(f'halo_{halo_id}_R500.fits', img_crop, hdr_crop, overwrite=True)
    return img_crop, hdr_crop

In [ ]:
def Plot_Crop_img(halo_index, hdr_crop, img_crop):
    fig, ax = plt.subplots(subplot_kw={'projection': wcs.WCS(hdr_crop)}, figsize=(4,4))

    norm = simple_norm(img_crop, 'log')

    im   = ax.imshow(img_crop, norm=norm, origin='lower', cmap='afmhot')

    ra  = ax.coords['ra']     
    dec = ax.coords['dec']    

    ra.set_format_unit(u.deg)         
    dec.set_format_unit(u.deg)

    ra.set_major_formatter('d.dd')     
    dec.set_major_formatter('d.dd')

    ax.set_xlabel('RA (deg)')
    ax.set_ylabel('Dec (deg)')
    ax.set_title(f'Halo {FOF_Halo_IDs[halo_index]} (kpc)')

    plt.tight_layout(); plt.show()

In [ ]:
def Get_Xray_Peak(hdr, img, sig=0):
    h = sim_info['hubble']
    z= redshift 

    wcs_info = wcs.WCS(hdr)

    img_s = img if sig == 0 else gaussian_filter(img, sigma=sig)

    y_peak, x_peak = np.unravel_index(np.nanargmax(img_s), img_s.shape)
    # print(y_peak, x_peak)
    # peak_counts    = img_s[y_peak, x_peak]

    # conver to ra and dec
    ra_deg, dec_deg = wcs_info.pixel_to_world_values(x_peak, y_peak)
    print(ra_deg, dec_deg)

    return ra_deg, dec_deg

In [ ]:
def Plot_Snap_Gas(halo_index, proj_axis='z', nbins =256, log_norm=True):
    halo_id = FOF_Halo_IDs[halo_index]
    subhalo_id = SubhaloIDs[halo_index]
    snap_cutout = snap_basic_path + f'cutout_sub{subhalo_id}_FOF{halo_id}.hdf5'

    with h5py.File(snap_cutout) as f:
        Masses = f['PartType0/Masses'][:]
        Coordinates = f['PartType0/Coordinates'][:]

    cx, cy, cz = GroupPos[halo_index]
    dx = Coordinates[:, 0] - cx
    dy = Coordinates[:, 1] - cy
    dz = Coordinates[:, 2] - cz

    if proj_axis == 'z':
        x, y = dx, dy
        xlabel, ylabel = r'$\Delta x\ ({\rm ckpc}/h)$', r'$\Delta y$'
    elif proj_axis == 'y':
        x, y = dz, dx
        xlabel, ylabel = r'$\Delta x$', r'$\Delta z$'
    elif proj_axis == 'x':
        x, y = dy, dz
        xlabel, ylabel = r'$\Delta y$', r'$\Delta z$'
    else:
        raise ValueError("proj_axis must be 'x','y' or 'z'.")
    
    H, xe, ye = np.histogram2d(x, y, bins=nbins,
                               weights=Masses)
    # H 单位：Msun/h

    # -------------------- 绘图 --------------------
    fig, ax = plt.subplots(figsize=(5, 5))
    norm = LogNorm() if log_norm else None

    im = ax.imshow(H.T,          # 注意转置
                   origin='lower',
                   norm=norm,
                   cmap='inferno',
                   interpolation='nearest')

    ax.set_aspect('equal')
    ax.set_xlabel(f'{xlabel} (ckpc/h)')
    ax.set_ylabel(f'{ylabel} (ckpc/h)')
    ax.set_title(f'Halo {halo_id}  |  Gas Mass Map  |  proj {proj_axis}')

    cbar = fig.colorbar(im, ax=ax, pad=0.01)
    cbar.set_label(r'$\Sigma M_{\rm gas}\ ({\rm M_\odot}/h)$')

    plt.tight_layout()
    plt.show()
    return im

In [ ]:
def Read_Bh_pos(halo_index,  proj_axis='z'):
    h = sim_info['hubble']
    z = redshift
    halo_id = FOF_Halo_IDs[halo_index]
    subhalo_id = SubhaloIDs[halo_index]
    bh_cutout = BH_basic_path + f'cutout_sub{subhalo_id}_FOF{halo_id}.hdf5'

    with h5py.File(bh_cutout) as f:
        BH_Mass = f['PartType5/BH_Mass'][:]
        # Masses = f['PartType5/Masses'][:]
        BH_Coordinates = f['PartType5/Coordinates'][:]

    cx, cy, cz = GroupPos[halo_index]        # ckpc/h
    # Postions of SMBH
    bx, by, bz = BH_Coordinates[np.argmax(BH_Mass)]

    dx_ckpch = bx - cx          # ckpc/h
    dy_ckpch = by - cy
    dz_ckpch = bz - cz

    dx_kpc = dx_ckpch *factor_a/ h
    dy_kpc = dy_ckpch * factor_a/ h
    dz_kpc = dz_ckpch *factor_a/ h

    if proj_axis == 'z':
        dra_kpc  = dx_kpc   # x → RA
        ddec_kpc = dy_kpc   # y → Dec
    elif proj_axis == 'x':
        dra_kpc  = dy_kpc   # y → RA
        ddec_kpc = dz_kpc   # z → Dec
    elif proj_axis == 'y':
        dra_kpc  = dz_kpc   # x → RA
        ddec_kpc = dx_kpc   # z → Dec

    # 3) convert to ra and dec
    DA = cosmo.angular_diameter_distance(z)   # Mpc
    dra  = (dra_kpc * u.kpc / DA).to(u.deg, equivalencies=u.dimensionless_angles())
    ddec = (ddec_kpc * u.kpc / DA).to(u.deg, equivalencies=u.dimensionless_angles())

    ra_bh_deg  = -dra.value
    dec_bh_deg = ddec.value

    return ra_bh_deg, dec_bh_deg

In [ ]:
def Cal_Offset(ra_deg, dec_deg,ra_bh_deg, dec_bh_deg, z):
    c_peak   = SkyCoord(ra_deg*u.deg,  dec_deg*u.deg,  frame='icrs')
    c_center = SkyCoord(ra_bh_deg*u.deg, dec_bh_deg*u.deg, frame='icrs')

    theta = c_peak.separation(c_center)  # Quantity, default rad

    DA = cosmo.angular_diameter_distance(z)
    R_kpc = (theta * DA).to(u.kpc,
            equivalencies=u.dimensionless_angles())
    
    return R_kpc

In [ ]:
img_basic_path = '/users_path/merger_trace/data/tng_cluster/tng_cluster_Xray_img/'
snap_basic_path = '/users_path/merger_trace/data/tng_cluster/tng_cluster_cutouts/tng_cluster_cutouts_gas/'
BH_basic_path = '/users_path/merger_trace/data/tng_cluster/tng_cluster_cutouts/tng_cluster_cutouts_bh/'

In [ ]:
sigma_list = [0, 3, 5, 10]
R_kpcs_all = np.zeros((len(FOF_Halo_IDs), len(sigma_list))) 
#for halo_index in range(len(FOF_Halo_IDs)-8,len(FOF_Halo_IDs)-7):
for halo_index in range(len(FOF_Halo_IDs)):
    hdr, img = Read_imgf(halo_index)

    img_crop, hdr_crop = Crop_img(hdr, img, redshift, halo_index)

    Plot_Crop_img(halo_index, hdr_crop, img_crop)
    Plot_Snap_Gas(halo_index, proj_axis='y', nbins =256, log_norm=True)

    for j, sigma in enumerate(sigma_list):
        ra_deg, dec_deg = Get_Xray_Peak(hdr_crop, img_crop, sig=sigma)
    
        ra_bh_deg, dec_bh_deg = Read_Bh_pos(halo_index, proj_axis='y')
        print(ra_bh_deg, dec_bh_deg)
    
        R_kpc = Cal_Offset(ra_deg, dec_deg,ra_bh_deg, dec_bh_deg, redshift)
    
        R_kpcs_all[halo_index, j] = R_kpc.value 
        print(R_kpc)

In [ ]:
df = pd.DataFrame(R_kpcs_all, columns=[f'sigma_{s}_projecty' for s in sigma_list])
df.insert(0, 'Subhalo_ID', SubhaloIDs)
df.insert(0, 'FOF_Halo_ID', FOF_Halo_IDs)

df.to_csv('BCG_offset_matrix_y.csv', index=False)
print(df.head())